In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pyspark

In [ ]:
# Team 9
Quách Bảo Hưng
Phạm Trung Kiên
Nguyễn Phi Long
Nguyễn Duy Phương

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f 

spark = SparkSession.builder.appName("PySparkKaggle").getOrCreate()

In [ ]:
#cau 1
ratings = (
    spark.read.csv(
        path="/kaggle/input/msis405l21cttt-spring-2021-assignment-7/movies_small.csv",
        sep=",",
        header=True,
        quote='"',
        schema="movieId INT, title STRING, genres STRING",
    )
)
ratings.printSchema()
ratings.filter(ratings.genres.like("%Action%")).show(5)

In [ ]:
#cau 2
ratings = (
    spark.read.csv(
        path="/kaggle/input/msis405l21cttt-spring-2021-assignment-7/movies_small.csv",
        sep=",",
        header=True,
        quote='"',
        schema="movieId INT, title STRING, genres STRING",
    )
)
cau2 = ratings.withColumn('movie',f.split('genres','\|')).withColumn('count',f.size('movie')).select('movieId','title','count')
cau2.show(5)

In [ ]:
#cau 3
ratings = (
    spark.read.csv(
        path="/kaggle/input/msis405l21cttt-spring-2021-assignment-7/movies_small.csv",
        sep=",",
        header=True,
        quote='"',
        schema="movieId INT, title STRING, genres STRING",
    )
)
ratings.withColumn('arr', f.split('genres', '\|')).withColumn('genres', f.explode('arr')).groupBy('genres').count().sort('count').show(5)

In [ ]:
#cau 4
ratings = (
    spark.read.csv(
        path="/kaggle/input/msis405l21cttt-spring-2021-assignment-7/movies_small.csv",
        sep=",",
        header=True,
        quote='"',
        schema="movieId INT, title STRING, genres STRING",
    )
)
ratings.withColumn('arr', f.split('genres', '\|')).withColumn('genres', f.explode('arr')).groupBy('genres').agg(f.collect_list('title'),).show(truncate=100)

In [ ]:
#cau 5
ratings = (
    spark.read.csv(
        path="/kaggle/input/msis405l21cttt-spring-2021-assignment-7/movies_small.csv",
        sep=",",
        header=True,
        quote='"',
        schema="movieId INT, title STRING, genres STRING",
    )
)
ratings.withColumn('arr', f.split('title', ' ')).withColumn('last_element', f.element_at('arr', -1)).withColumn('year', f.substring('last_element', 2,4).cast('int')).withColumn('arr_genres', f.split('genres', '\|')).withColumn('genres', f.explode('arr_genres')).groupBy('genres').agg(f.min('year').alias('first_appearance')).filter(f.substring('genres',0,10).isin(['Sci-Fi','Animation'])).select(f.col('genres').alias('genre'), 'first_appearance').show(1000, truncate=False)